# Hashing it all out

- https://adventofcode.com/2023/day/15

We are creating a simple [hash function](https://en.wikipedia.org/wiki/Hash_function). Because there are only 256 possible outputs for the function, and there are only [128 possible ASCII codepoints](https://en.wikipedia.org/wiki/ASCII), I've choosen to create a small table of 128 + 256 = 384 possible outputs for the new value of the hash after adding the next ASCII number. The table can then replace the multication and remainder operations.

To map the characters of the input string to their ASCII codepoints, just encode the string to an ASCII `bytes` value; iteration over a Python `bytes` object gives you the integer byte values, which are the ASCII codepoints we wanted.

As a final nod to performance, I'm storing the lookup table as a private keyword argument to the function, so that Python can look it up as a local variable. This is faster than looking up global variables. Perhaps part two will up the ante, requiring hashing huge strings, and then every nanosecond counts. :D As it stands, on my laptop the function now takes about 145 nanoseconds to hash the word `"HASH"`. Not bad for Python code!


In [1]:
import typing as t

_TABLE: t.Final[tuple[int, ...]] = tuple((i * 17) % 256 for i in range(256 + 128))


def holiday_ascii_string_helper(s: str, _t: tuple[int, ...] = _TABLE) -> int:
    hash = 0
    for c in s.encode("ascii"):
        hash = _t[hash + c]
    return hash


assert holiday_ascii_string_helper("HASH") == 52
test_steps = str("rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7").split(",")
assert sum(map(holiday_ascii_string_helper, test_steps)) == 1320

In [2]:
import aocd

steps = aocd.get_data(day=15, year=2023).strip().split(",")
print("Part 1:", sum(map(holiday_ascii_string_helper, steps)))

Part 1: 511343


# It's a dictionary!

For part two, we are going to implement a real [hashmap](https://en.wikipedia.org/wiki/Hashmap); or, as Python calls it, the [_dictionary type_](https://docs.python.org/3/library/stdtypes.html#dict), aka `dict`. In a hashmap, the boxes are commonly referred to as 'buckets'; a hashing algorithm selects what bucket to store key-value pairs into (or just to find the value for a given key). Because multiple input values can hash to the same bucket, you need a way to handle _collisions_, a way to store multiple values in the same bucket.

This is what happens inside the boxes here, it's a [collision resolution scheme](https://en.wikipedia.org/wiki/Hash_table#Collision_resolution), and here we are using _separate chaining_ to put multiple values into the same bucket. The Python dictionary type uses a different resolution scheme, it uses [_open addressing_](https://en.wikipedia.org/wiki/Hash_table#Open_addressing), but the idea is the same. When you look up a key to find the value, you use the hash function to find the corresponding bucket and then use equality tests for each key there in turn until you have the correct one.

Since Python 3.6 Python's dictionaries [preserve insertion order](https://stackoverflow.com/a/39537308/100297), and since Python 3.7, this fact was enshrined in the Python language specification. This means that when you use `dictonary[key] = value` to set a value in the dictionary, it'll add that key to the 'end' of the ordering, _unless_ the key is already in the table, at which point it'll just keep the same position. Looping over the keys or values of the dictionary then produces those keys or values in the order they were inserted. That's very handy here, we just make our boxes Python dictionaries, and so avoid having to test every key in each box to see if there already is a given label in that box.


In [3]:
import re

_instr = re.compile(r"([-=])")


def hashmap(steps: list[str]) -> int:
    boxes: tuple[dict[str, int], ...] = tuple({} for _ in range(256))
    for step in steps:
        label, instr, lens = _instr.split(step)
        box = boxes[holiday_ascii_string_helper(label)]
        if instr == "-":
            box.pop(label, None)
        else:
            box[label] = int(lens)
    return sum(
        b * l * lens
        for b, box in enumerate(boxes, 1)
        for l, lens in enumerate(box.values(), 1)
    )


assert hashmap(test_steps) == 145

In [4]:
print("Part 2:", hashmap(steps))

Part 2: 294474
